# L5: Business Process Agent

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Setup

In [ ]:
from helper import get_openai_api_key
from helper import parameterize_sparql

from langchain_openai import OpenAIEmbeddings, ChatOpenAI

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>
</div>

In [ ]:
openai_api_key = get_openai_api_key()

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    max_retries=2)

In [ ]:
from rdflib import Dataset

graph = Dataset(default_union=True)
graph.parse("./ro_shared_data/odata_knowledge_graph.ttl",
            format="turtle")

## Fetch API specifications

In [ ]:
from helper import fetch_entity_specification

In [ ]:
fetch_entity_specification(graph,
    entity_set_uri="http://data.example.org/Service/API_PURCHASEORDER_2/EntitySet/PURCHASEORDER"
)

## Discovery Tool

In [ ]:
import pickle

with open("../ro_shared_data/entity_sets_index.pickle", "rb") as file:
    index = pickle.load(file)

with open("../ro_shared_data/entity_set_uris.pickle", "rb") as file:
    entity_set_uris = pickle.load(file)

In [ ]:
from helper import discover_apis_and_process

In [ ]:
query = """Show me the active purchase orders in
purchasing group 002 and purchasing organization 3000"""
discover_apis_and_process(
    query=query,
    graph=graph,
    index=index,
    entity_set_uris=entity_set_uris,
    embedding_model=embedding_model,
)

In [ ]:
from langchain_core.tools import tool

@tool
def discover_apis(user_query: str) -> dict:
    """Discovers relevant API specifications and process information
    based on the user query.

    Args:
            user_query (str): user query

    Returns:
            discovery: (dict) of the form {'api_specs': api_specs,
            'process_information': process_information}
    """
    discovery = discover_apis_and_process(
        query=user_query,
        graph=graph,
        index=index,
        entity_set_uris=entity_set_uris,
        embedding_model=embedding_model,
    )
    api_specs = [
        fetch_entity_specification(graph ,entity_set_uri)
        for entity_set_uri in discovery["entity_sets"]
    ]
    return {
        "api_specs": api_specs,
        "process_information": discovery["process_information"],
    }

## Tools for calling mocked Data APIs

In [ ]:
from helper import (data_prs, data_pos, data, 
                    post_data_mock, get_data_mock, 
                    display_pos, display_prs)

In [ ]:
display_prs(data_prs)

In [ ]:
display_pos(data_pos)

In [ ]:
post_data_mock(
    data=data,
    service_name="API_PURCHASEORDER_2",
    entity_set="PURCHASEORDER",
    payload={
        "PurchaseOrderItem": [
            {"Material": "mouse", "OrderQuantity": 5},
            {"Material": "keyboard", "OrderQuantity": 3},
        ]
    },
)
display_pos(data_pos)

In [ ]:
get_data_mock(
    data=data,
    service_name="API_PURCHASEORDER_2",
    entity_set="PURCHASEORDER",
    filter_string="""PurchasingGroup eq '005'
    and PurchasingOrganization eq '3000'""",
    selects_string="PurchaseOrder,PurchasingGroup,PurchasingOrganization",
)


In [ ]:
@tool
def get_api(service_name: str, entity_set: str,
    filter_string: str | None = None, selects_string: str | None = None,
) -> dict:
    """Fetches data from an API based on the provided parameters.

    Args:
            service_name (str): Name of the service.
            entity_set (str): Name of the entity set to query.
            filter_string (str, optional): Filter conditions for the query. Defaults to None.
            selects_string (str, optional): Fields to select in the response. Defaults to None.

    Returns:
            dict: The response from the API.
    """
    return get_data_mock(data=data, service_name=service_name,
        entity_set=entity_set, filter_string=filter_string, 
        selects_string=selects_string,
    )

@tool
def post_data_api(service_name: str, entity_set: str, 
                  payload: dict | None = None) -> dict:
    """Posts data to an API based on the provided parameters.

    Args:
            service_name (str): Name of the service.
            entity_set (str): Name of the entity set to post data to.
            payload (dict): The data to be posted.

    Returns:
            dict: The response from the API.
    """
    return post_data_mock(
        data=data, service_name=service_name, entity_set=entity_set, payload=payload
    )

## Define Agent

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful assistant that can discover and
            call OData APIs to serve user requests.""",
        ),
        (
            "human",
            """
            1. Discover the relevant APIs and process information based
            on the user query. Use the full user query to discover the
            APIs.
            2. If you discover business process information relevant to
            the user query, make sure to follow the process to serve
            the user request.
            3. If an entityset has navigations, these can be used to
            create parent and child object in one API call to the parent.
            The payload may look like this:
            {{
                "parent_property_1": "value_1",
                "parent_property_2": "value_2",
                "child_entity_set_name": [
                {{"child_property_1": "value_3"}},
                {{"child_property_2": "value_4"}}
                ]
            }}

			- DO NOT create child objects without creating the parent 
              object first. 
            - Pay attention to the cardinality of the navigations. 
              If the cardinality is one_to_many, the dependent entity set
              can be a list of objects.
			- Do not invent values, for properties that are not specified,
              to not include them in the payload.        
		    - Use all fields that you're given values for 
              (either by the user or from previous tool calls) which match
              API properties in the payload.
            - If you create objects, do not include the key properties 
              in the payload. The keys properties are generated by 
              the backend.
			- When referring to previously created objects,
              use the keys of the objects in the payload 
			- If you create objects, do not query them to check.
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [ ]:
agent = create_tool_calling_agent(
    llm=llm, tools=[discover_apis, get_api, post_data_api], prompt=prompt
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=[get_api, post_data_api, discover_apis],
    verbose=True,
    return_intermediate_steps=True,
)

## Example 1: Get active Purchase Orders

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Note about LLMs and Agent State of the Art, Please Read!:</b>   LLMs do not always produce the same results. Consequently, the agent behavior you experience when running labs may differ from the video. AI agents are at the cutting edge of development. Managing probablistic behavior is an ongoing area of research.</p>

In [ ]:
res = agent_executor.invoke(
    {
        "input": """Show me active purchase orders in
        purchasing group 002 and purchasing organization 3000"""
    }
)

In [ ]:
fixed_values_insert = """
PREFIX odata: <http://example.org/odata#>
INSERT DATA {
    <http://data.example.org/Service/API_PURCHASEORDER_2/EntityType/PURCHASEORDER_TYPE/Property/PURCHASINGPROCESSINGSTATUS> odata:valueHelp 
        [ odata:key "01" ; odata:value "In process" ],
        [ odata:key "02" ; odata:value "Active" ],
        [ odata:key "03" ; odata:value "In release" ],
        [ odata:key "04" ; odata:value "Partially released" ],
        [ odata:key "05" ; odata:value "Release completed" ],
        [ odata:key "08" ; odata:value "Rejected" ] .
}
"""
graph.update(fixed_values_insert)

In [ ]:
res = agent_executor.invoke(
    {
        "input": """Show me the active purchase orders in
        purchasing group 002 and purchasing organization 3000"""
    }
)

## Example 2: Create Purchase Order

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Note about LLMs and Agent State of the Art, Please Read!:</b>   LLMs do not always produce the same results. Consequently, the agent behavior you experience when running labs may differ from the video. AI agents are at the cutting edge of development. Managing probablistic behavior is an ongoing area of research.</p>

In [ ]:
res = agent_executor.invoke(
    {
        "input": """Create a purchase order for 5 pencils in
        purchasing group 002 and purchasing organization 3000"""
    }
)

In [ ]:
display_prs(data_prs)

In [ ]:
display_pos(data_pos)